In [1]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
train_augmentation = ImageDataGenerator(rescale=1./255, 
									rotation_range=40, 
									width_shift_range=0.2, 
									height_shift_range=0.2,
            						shear_range=0.2, 
            						zoom_range=0.2, 
            						horizontal_flip=True, 
            						fill_mode='nearest')                  
val_augmentation = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_augmentation.flow_from_directory('data/images/train/', 
										target_size=(128, 128), 
										batch_size=32, 
            							class_mode='binary',
										color_mode='grayscale')

val_generator = val_augmentation.flow_from_directory('data/images/val/', 
										target_size=(128, 128), 
										batch_size=32,
            							class_mode='binary',
										color_mode='grayscale')

In [ ]:
train_generator.class_indices

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [7]:
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [8]:
checkpoint = 'model/model-{epoch:03d}.h5'
model_checkpoint = ModelCheckpoint(filepath=checkpoint,
                                save_weights_only=False,
                                verbose=1,
                                monitor='val_accuracy',
                                mode='auto',
                                save_freq=1000)

In [ ]:
history = model.fit(train_generator, 
                steps_per_epoch=100,
                epochs=100, 
                validation_data=val_generator, 
                validation_steps=50, 
                callbacks=model_checkpoint)